In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/data.csv')

C:\Users\zyygf\AppData\Local\Temp\ipykernel_29236\269992028.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/data.csv')


In [3]:
useful_columns = ['skill', 'evaluation_code', 'attack_code', 'serving_team', 'team', 'point']

In [4]:
temp_df = df[useful_columns]
temp_df

,skill,evaluation_code,attack_code,serving_team,team,point
0,NaN,NaN,NaN,University of Iowa,University of Iowa,False
1,NaN,NaN,NaN,University of Iowa,University of Iowa,False
2,NaN,NaN,NaN,University of Iowa,Ohio State University,False
3,NaN,NaN,NaN,University of Iowa,Ohio State University,False
4,Serve,-,NaN,University of Iowa,University of Iowa,False
...,...,...,...,...,...,...
1328919,Set,#,NaN,University of Oklahoma,Texas Christian University,False
1328920,Attack,#,X5,University of Oklahoma,Texas Christian University,False
1328921,Dig,=,NaN,University of Oklahoma,University of Oklahoma,False
1328922,NaN,NaN,NaN,University of Oklahoma,Texas Christian University,True


In [5]:
temp_df['side'] = temp_df.apply(lambda x: 'S' if x['serving_team'] == x['team'] else 'R' if pd.notna(x['team']) else None, axis=1)
temp_df

C:\Users\zyygf\AppData\Local\Temp\ipykernel_29236\4159250422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['side'] = temp_df.apply(lambda x: 'S' if x['serving_team'] == x['team'] else 'R' if pd.notna(x['team']) else None, axis=1)


,skill,evaluation_code,attack_code,serving_team,team,point,side
0,NaN,NaN,NaN,University of Iowa,University of Iowa,False,S
1,NaN,NaN,NaN,University of Iowa,University of Iowa,False,S
2,NaN,NaN,NaN,University of Iowa,Ohio State University,False,R
3,NaN,NaN,NaN,University of Iowa,Ohio State University,False,R
4,Serve,-,NaN,University of Iowa,University of Iowa,False,S
...,...,...,...,...,...,...,...
1328919,Set,#,NaN,University of Oklahoma,Texas Christian University,False,R
1328920,Attack,#,X5,University of Oklahoma,Texas Christian University,False,R
1328921,Dig,=,NaN,University of Oklahoma,University of Oklahoma,False,S
1328922,NaN,NaN,NaN,University of Oklahoma,Texas Christian University,True,R


In [6]:
temp_df.loc[temp_df['point'] == True, ['skill', 'evaluation_code']] = ['Point', '']

In [7]:
skill_code_dict = {
    'Serve': 'SV',
    'Reception': 'R', 
    'Set': 'S', 
    'Attack': 'A', 
    'Dig': 'D', 
    'Point': 'P', 
    'Block': 'B', 
    'Freeball': 'F'
}
temp_df['skill_code'] = temp_df['skill'].map(skill_code_dict)

C:\Users\zyygf\AppData\Local\Temp\ipykernel_29236\726817799.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['skill_code'] = temp_df['skill'].map(skill_code_dict)


In [8]:
temp_df.loc[temp_df['skill_code'].isna(), 'side'] = np.nan
temp_df.loc[(temp_df['attack_code'].isna()) & (temp_df['skill_code'] == 'A'), 'attack_code'] = 'NA'

In [9]:
def create_short_state_code(df):
    # Initialize the 'short_state' and 'prev_state' columns with NaN values
    df['short_state'] = np.nan
    df['prev_state'] = np.nan
    
    # Variable to store the previous state
    previous_state = None
    
    # Iterate over the dataframe row by row
    for i in range(len(df)):
        skill_code = df.at[i, 'skill_code']
        side = df.at[i, 'side']
        evaluation_code = df.at[i, 'evaluation_code']
        attack_code = df.at[i, 'attack_code']
        
        # If skill_code is NaN, this is the separator between points
        if pd.isna(skill_code):
            previous_state = None
            continue

        # Rule 2: If the skill_code is 'SV' (serve), start a new state code with 'S_SV'
        if skill_code == 'SV':
            df.at[i, 'prev_state'] = previous_state  # Store previous state
            df.at[i, 'short_state'] = f"{side}_SV"
            previous_state = df.at[i, 'short_state']
            continue

        # For attacks, append attack_code, no evaluation_code
        if skill_code == 'A':
            df.at[i, 'prev_state'] = previous_state  # Store previous state
            df.at[i, 'short_state'] = f"{side}_A{attack_code}"
            previous_state = df.at[i, 'short_state']
            continue

        # For other skills, append evaluation_code
        df.at[i, 'prev_state'] = previous_state  # Store previous state
        df.at[i, 'short_state'] = f"{side}_{skill_code}{evaluation_code}"
        previous_state = df.at[i, 'short_state']

    return df

In [10]:
def remove_consecutive_nan_rows(df):
    # Identify rows where all values are NaN
    nan_rows = df['skill_code'].isna()

    # Create a mask to mark the start of each block of consecutive NaNs
    drop_mask = nan_rows & nan_rows.shift(fill_value=False)

    # Remove consecutive NaN rows but keep the first one
    df_cleaned = df[~drop_mask].copy()

    return df_cleaned

# Apply the function to clean the DataFrame
temp_df = remove_consecutive_nan_rows(temp_df)
temp_df = temp_df.reset_index(drop=True)

In [12]:
state_df = create_short_state_code(temp_df)
state_df

C:\Users\zyygf\AppData\Local\Temp\ipykernel_29236\3885978384.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'S_SV' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'short_state'] = f"{side}_SV"
C:\Users\zyygf\AppData\Local\Temp\ipykernel_29236\3885978384.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'S_SV' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'prev_state'] = previous_state  # Store previous state


,skill,evaluation_code,attack_code,serving_team,team,point,side,skill_code,short_state,prev_state
0,NaN,NaN,NaN,University of Iowa,University of Iowa,False,NaN,NaN,NaN,NaN
1,Serve,-,NaN,University of Iowa,University of Iowa,False,S,SV,S_SV,NaN
2,Reception,+,NaN,University of Iowa,Ohio State University,False,R,R,R_R+,S_SV
3,Set,#,NaN,University of Iowa,Ohio State University,False,R,S,R_S#,R_R+
4,Attack,#,X6,University of Iowa,Ohio State University,False,R,A,R_AX6,R_S#
...,...,...,...,...,...,...,...,...,...,...
953854,Set,#,NaN,University of Oklahoma,Texas Christian University,False,R,S,R_S#,R_R#
953855,Attack,#,X5,University of Oklahoma,Texas Christian University,False,R,A,R_AX5,R_S#
953856,Dig,=,NaN,University of Oklahoma,University of Oklahoma,False,S,D,S_D=,R_AX5
953857,Point,,NaN,University of Oklahoma,Texas Christian University,True,R,P,R_P,S_D=


In [14]:
def create_transition_matrix(df, state_column, prev_state_column):
    # Filter rows where both 'prev_state' and 'state' are not NaN
    valid_transitions = df.dropna(subset=[state_column])

    # Count occurrences of each transition (prev_state -> state)
    transition_counts = valid_transitions.groupby([prev_state_column, state_column]).size().unstack(fill_value=0)

    # Get the full set of unique states from both indices (prev_state) and columns (state)
    all_states = sorted(set(transition_counts.index).union(set(transition_counts.columns)))

    # Reindex the matrix to ensure all states are present in both rows and columns
    transition_matrix = transition_counts.reindex(index=all_states, columns=all_states, fill_value=0)

    # Normalize counts to get probabilities (empirical transition matrix)
    transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0).fillna(0)

    return transition_matrix

# Apply the function to the dataframe
transition_matrix = create_transition_matrix(state_df, 'short_state', 'prev_state')

In [15]:
transition_matrix

short_state,R_ACB,R_ACD,R_ACF,R_ANA,R_APP,R_AV0,R_AV5,R_AV6,R_AV8,R_AVB,...,S_F/,S_F=,S_P,S_R-,S_R/,S_R=,S_S#,S_S-,S_S=,S_SV
prev_state,,,,,,,,,,,,,,,,,,,,,
R_ACB,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.091298,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACD,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.085981,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ANA,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000181,0.000181,0.090054,0.000181,0.000000,0.000000,0.000362,0.000000,0.0,0.0
R_APP,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000541,0.000000,0.000000,0.000000,0.000541,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_R=,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S#,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S-,0.0,0.0,0.0,0.089571,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000818,0.048671,0.010634,0.000000,0.000000,0.000000,0.000818,0.000818,0.0,0.0


In [16]:
def calculate_absorption_probabilities(transition_matrix, absorbing_states):
    # transient and absorbing states
    all_states = list(transition_matrix.index)
    transient_states = [state for state in all_states if state not in absorbing_states]

    # Extract submatrices Q and R
    Q = transition_matrix.loc[transient_states, transient_states].values
    R = transition_matrix.loc[transient_states, absorbing_states].values

    # Calculate N = (I - Q)^(-1)
    I = np.eye(len(Q))
    N = np.linalg.inv(I - Q)

    # Calculate absorption probabilities B = N * R
    B = np.dot(N, R)

    # Convert results to a DataFrame
    absorption_probabilities = pd.DataFrame(B, index=transient_states, columns=absorbing_states)
    
    return absorption_probabilities, N

In [17]:
absorbing_states = ["R_P", "S_P"]
absorption_probabilities, N = calculate_absorption_probabilities(transition_matrix, absorbing_states)

In [18]:
absorption_probabilities.loc[['S_SV']]

,R_P,S_P
S_SV,0.579049,0.420951
